In [1]:
import numpy as np
import pandas as pd
import sklearn

import lightgbm as lgb
from xgboost import XGBClassifier

from sklearn.linear_model import LogisticRegression
from sklearn import metrics

import statsmodels.api as sm
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings('ignore')

%load_ext autotime

time: 161 µs (started: 2022-05-08 16:47:18 +04:00)


In [2]:
filepath = "../Data/TrainData0.csv"

train_data = pd.read_csv(filepath)

filepath = "../Data/TestData0.csv"

test_data = pd.read_csv(filepath)

time: 2.9 s (started: 2022-05-08 16:47:18 +04:00)


In [3]:
# Dropping "Lead Flag" and "ZIPCode" to run LightGBM model on the train and test datasets

x_train = train_data.drop(['Lead Flag', 'ZIPCode'], axis=1)
y_train = train_data['Lead Flag']

x_test = test_data.drop(['Lead Flag', 'ZIPCode'], axis=1)
y_test = test_data['Lead Flag']

time: 266 ms (started: 2022-05-08 16:47:21 +04:00)


In [4]:
# In this dictionary the results of LightGBM will be added, then dataframe will be made based on this

compare_results_lgbm = {'learning_rate':[], 'max_depth':[], 'score_train':[], 'score_test': []}

time: 234 µs (started: 2022-05-08 16:47:21 +04:00)


In [5]:
# Running nested for loops to put values in the parameters of LightGBM, run the model, save the train and test scores. 

results = []
for i in np.arange(0.1, 0.9, 0.1):
    for j in range(0,5, 1):
        
        model = lgb.LGBMClassifier(learning_rate=i, max_depth=j, random_state=42)
        model.fit(x_train, y_train, eval_set=[(x_train, y_train)], eval_metric='AUC', verbose=20)
        
        compare_results_lgbm['learning_rate'].append(i)
        compare_results_lgbm['max_depth'].append(j)
        
        Y0_lgbm=model.predict_proba(x_train)[:,1]
        Y1_lgbm=model.predict_proba(x_test)[:,1]
        
        compare_results_lgbm['score_train'].append(roc_auc_score(y_train, Y0_lgbm))
        compare_results_lgbm['score_test'].append(roc_auc_score(y_test, Y1_lgbm))


[20]	training's auc: 0.733022	training's binary_logloss: 0.0497204
[40]	training's auc: 0.77015	training's binary_logloss: 0.0476945
[60]	training's auc: 0.796037	training's binary_logloss: 0.0461141
[80]	training's auc: 0.815317	training's binary_logloss: 0.0448151
[100]	training's auc: 0.831934	training's binary_logloss: 0.043588
[20]	training's auc: 0.667377	training's binary_logloss: 0.0529871
[40]	training's auc: 0.680512	training's binary_logloss: 0.0524929
[60]	training's auc: 0.688623	training's binary_logloss: 0.0521995
[80]	training's auc: 0.693829	training's binary_logloss: 0.0519939
[100]	training's auc: 0.697523	training's binary_logloss: 0.0518394
[20]	training's auc: 0.687254	training's binary_logloss: 0.0523098
[40]	training's auc: 0.702092	training's binary_logloss: 0.0517355
[60]	training's auc: 0.709459	training's binary_logloss: 0.051426
[80]	training's auc: 0.715353	training's binary_logloss: 0.0512177
[100]	training's auc: 0.718973	training's binary_logloss: 0.051

[80]	training's auc: 0.76234	training's binary_logloss: 0.0497375
[100]	training's auc: 0.773101	training's binary_logloss: 0.0489234
[20]	training's auc: 0.712507	training's binary_logloss: 0.0603574
[40]	training's auc: 0.712707	training's binary_logloss: 0.0619432
[60]	training's auc: 0.714063	training's binary_logloss: 0.0659184
[80]	training's auc: 0.710935	training's binary_logloss: 0.0698029
[100]	training's auc: 0.708597	training's binary_logloss: 0.068571
[20]	training's auc: 0.69632	training's binary_logloss: 0.0516743
[40]	training's auc: 0.70785	training's binary_logloss: 0.0513023
[60]	training's auc: 0.712264	training's binary_logloss: 0.051147
[80]	training's auc: 0.715193	training's binary_logloss: 0.0510514
[100]	training's auc: 0.717321	training's binary_logloss: 0.0509834
[20]	training's auc: 0.71065	training's binary_logloss: 0.0512187
[40]	training's auc: 0.722055	training's binary_logloss: 0.0508486
[60]	training's auc: 0.728192	training's binary_logloss: 0.050652

In [6]:
# Making dataframe from dictionary. Getting the best test score for LightGBM and corresponding parameters.

compare_results_lgbm = pd.DataFrame.from_dict(compare_results_lgbm)
compare_results_lgbm[compare_results_lgbm['score_test'] == max(compare_results_lgbm['score_test'])] 

,learning_rate,max_depth,score_train,score_test
7,0.2,2,0.729615,0.715362


time: 21.4 ms (started: 2022-05-08 16:50:58 +04:00)


In [7]:
# In this dictionary the results of XGBoost will be added, then dataframe will be made based on this

compare_results_xgb = {'learning_rate':[], 'max_depth':[], 'score_train':[], 'score_test': []}

time: 265 µs (started: 2022-05-08 16:50:58 +04:00)


In [8]:
# Running nested for loops to put values in the parameters of XGBoost, run the model, save the train and test scores. 

results = []
for i in np.arange(0.1, 0.9,0.2):
    for j in range(1,10, 2):
        print('learning rate: ', i, " max_depth: ", j)
        model = XGBClassifier(learning_rate=i, max_depth=j, random_state=42)
        model.fit(x_train, y_train, eval_set=[(x_train, y_train)], eval_metric='auc', verbose=20)
        compare_results_xgb['learning_rate'].append(i)
        compare_results_xgb['max_depth'].append(j)
        
        
        Y0_xgb=model.predict_proba(x_train)[:,1]
        Y1_xgb=model.predict_proba(x_test)[:,1]
        
        compare_results_xgb['score_train'].append(roc_auc_score(y_train, Y0_xgb))
        compare_results_xgb['score_test'].append(roc_auc_score(y_test, Y1_xgb)) 

learning rate:  0.1  max_depth:  1
[0]	validation_0-auc:0.59582
[20]	validation_0-auc:0.59582
[40]	validation_0-auc:0.64900
[60]	validation_0-auc:0.66925
[80]	validation_0-auc:0.68202
[99]	validation_0-auc:0.68817
learning rate:  0.1  max_depth:  3
[0]	validation_0-auc:0.62986
[20]	validation_0-auc:0.67053
[40]	validation_0-auc:0.68094
[60]	validation_0-auc:0.70423
[80]	validation_0-auc:0.71919
[99]	validation_0-auc:0.72766
learning rate:  0.1  max_depth:  5
[0]	validation_0-auc:0.63057
[20]	validation_0-auc:0.69280
[40]	validation_0-auc:0.71563
[60]	validation_0-auc:0.74638
[80]	validation_0-auc:0.76608
[99]	validation_0-auc:0.77651
learning rate:  0.1  max_depth:  7
[0]	validation_0-auc:0.63092
[20]	validation_0-auc:0.71011
[40]	validation_0-auc:0.76966
[60]	validation_0-auc:0.81519
[80]	validation_0-auc:0.84005
[99]	validation_0-auc:0.85472
learning rate:  0.1  max_depth:  9
[0]	validation_0-auc:0.63109
[20]	validation_0-auc:0.72974
[40]	validation_0-auc:0.83663
[60]	validation_0-au

In [9]:
# Making dataframe from dictionary. Getting the best test score for XGBoost and corresponding parameters.

compare_results_xgb = pd.DataFrame.from_dict(compare_results_xgb)
compare_results_xgb[compare_results_xgb['score_test'] == max(compare_results_xgb['score_test'])] 

,learning_rate,max_depth,score_train,score_test
10,0.5,1,0.716266,0.714031


time: 4.11 ms (started: 2022-05-08 17:05:37 +04:00)


In [10]:
# Running Logistic Regression without regularization 

logreg = LogisticRegression(random_state=42)

logreg.fit(x_train,y_train)

Y0_logreg=logreg.predict_proba(x_train)[:,1]
Y1_logreg=logreg.predict_proba(x_test)[:,1]

print("ROC_AUC Train:", roc_auc_score (y_train, Y0_logreg))
print ("ROC_AUC Test:", roc_auc_score(y_test, Y1_logreg))

ROC_AUC Train: 0.544602749211782
ROC_AUC Test: 0.5438309032958163
time: 8.08 s (started: 2022-05-08 17:05:37 +04:00)


In [11]:
# Running Logistic Regression with regularization. Better results than without regularization

logreg=LogisticRegression(C=1,penalty='l1',solver="liblinear", random_state=42)

logreg.fit(x_train,y_train)

Y0_logreg=logreg.predict_proba(x_train)[:,1]
Y1_logreg=logreg.predict_proba(x_test)[:,1]

print("ROC_AUC Train:", roc_auc_score (y_train, Y0_logreg))
print ("ROC_AUC Test:", roc_auc_score(y_test, Y1_logreg))

ROC_AUC Train: 0.7048604020751943
ROC_AUC Test: 0.7063236361220072
time: 30.4 s (started: 2022-05-08 17:05:45 +04:00)
